## Importing Libraries

In [14]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
# Printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Preprocessing

In [11]:
dataset = pd.read_csv("D:/Projects/Fake News Prediction_4/train.csv")

In [12]:
dataset.shape

(20800, 5)

In [13]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
# counting the number of missing values in dataset
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [31]:
#Replacing null values with empty string
datset = dataset.fillna("")

In [32]:
dataset['content'] = datset['author'] + ' ' + dataset['title']

In [55]:
dataset['content'] = dataset['content'].astype(str)
print(dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [56]:
# Separating data and label
x = dataset.drop(columns = 'label', axis = 1)
y = dataset['label']

In [57]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

## Stemming

In [58]:
# it is the process of reducing a word to its root word
# ex: actor, actress, acting --> act

In [59]:
porter = PorterStemmer()

In [60]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [61]:
dataset['content'] = dataset['content'].apply(stemming)

In [62]:
print(dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [63]:
# separating the data and label
x = dataset['content'].values
y = dataset['label'].values

In [64]:
print(x)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


## Vectorizing

In [67]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x)

In [68]:
print(x)

  (0, 15440)	0.28485063562728646
  (0, 2452)	0.3676519686797209
  (0, 7586)	0.24785219520671603
  (0, 8488)	0.29212514087043684
  (0, 2921)	0.2468450128533713
  (0, 13263)	0.2565896679337957
  (0, 4910)	0.233316966909351
  (0, 264)	0.27010124977708766
  (0, 3745)	0.2705332480845492
  (0, 6912)	0.21874169089359144
  (0, 8764)	0.3635963806326075
  (0, 3555)	0.3598939188262559
  (1, 1870)	0.15520442784430935
  (1, 2195)	0.3826942772595605
  (1, 16540)	0.30068778691953485
  (1, 1480)	0.29396015040705736
  (1, 2778)	0.1909269013762726
  (1, 6728)	0.19048702556772562
  (1, 5435)	0.7142594577579127
  (1, 3524)	0.26405681003334164
  (2, 5322)	0.3866530551182615
  (2, 5892)	0.3474613386728292
  (2, 9463)	0.49351492943649944
  (2, 15368)	0.41544962664721613
  (2, 2905)	0.3179886800654691
  :	:
  (20797, 1275)	0.3348237293980069
  (20797, 12917)	0.24784045671633245
  (20797, 12153)	0.27218191633560185
  (20797, 14736)	0.31107718530764283
  (20797, 11951)	0.24737117918660476
  (20797, 9362)	0.2949

## splitting dataset into train and test

In [69]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [71]:
print(x.shape, x_train.shape, y_train.shape)

(20800, 16862) (16640, 16862) (16640,)


## Training Model

In [73]:
classifier = LogisticRegression()
classifier.fit(x_train,y_train)

LogisticRegression()

## Evaluation

In [74]:
# training data acc_score
x_train_pred = classifier.predict(x_train)
train_acc = accuracy_score(x_train_pred, y_train)

In [75]:
print(train_acc)

0.9884014423076923


In [76]:
# test data acc_score
x_test_pred = classifier.predict(x_test)
test_acc = accuracy_score(y_test, x_test_pred)

In [77]:
print(test_acc)

0.9795673076923077


In [78]:
x_new = x_test[3]

prediction = classifier.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [79]:
print(y_test[0])

0
